In [3]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np

In [6]:
xy = np.loadtxt('data-01-test-score.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

In [13]:
# Make sure the shape and data are OK
print(x_data.shape, x_data, len(x_data))
print(y_data.shape, y_data)

(25, 3) [[ 73.  80.  75.]
 [ 93.  88.  93.]
 [ 89.  91.  90.]
 [ 96.  98. 100.]
 [ 73.  66.  70.]
 [ 53.  46.  55.]
 [ 69.  74.  77.]
 [ 47.  56.  60.]
 [ 87.  79.  90.]
 [ 79.  70.  88.]
 [ 69.  70.  73.]
 [ 70.  65.  74.]
 [ 93.  95.  91.]
 [ 79.  80.  73.]
 [ 70.  73.  78.]
 [ 93.  89.  96.]
 [ 78.  75.  68.]
 [ 81.  90.  93.]
 [ 88.  92.  86.]
 [ 78.  83.  77.]
 [ 82.  86.  90.]
 [ 86.  82.  89.]
 [ 78.  83.  85.]
 [ 76.  83.  71.]
 [ 96.  93.  95.]] 25
(25, 1) [[152.]
 [185.]
 [180.]
 [196.]
 [142.]
 [101.]
 [149.]
 [115.]
 [175.]
 [164.]
 [141.]
 [141.]
 [184.]
 [152.]
 [148.]
 [192.]
 [147.]
 [183.]
 [177.]
 [159.]
 [177.]
 [175.]
 [175.]
 [149.]
 [192.]]


In [14]:
# placeholders for a tensor that will be always fed
X = tf.placeholder(tf.float32, shape=[None, 3])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([3, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

In [15]:
hypothesis = tf.matmul(X, W) + b

# Simplified cost/Loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

# Minimize
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

In [16]:
# Launch the graph in a session
sess = tf.Session()

#Initializes global variables in the graph
sess.run(tf.global_variables_initializer())

# Set up feed_dict variables inside the loop
for step in range(2001):
    cost_val, hy_val, _ = sess.run(
        [cost, hypothesis, train],
        feed_dict={X: x_data, Y: y_data})
    if step % 10 == 0:
        print(step, "Cost: ", cost_val,
             "\nPrediction:\n", hy_val)

0 Cost:  133116.44 
Prediction:
 [[-191.63998]
 [-226.50742]
 [-225.43845]
 [-244.3607 ]
 [-172.33296]
 [-123.98508]
 [-181.61885]
 [-133.14024]
 [-208.94879]
 [-190.06561]
 [-175.0196 ]
 [-170.17032]
 [-234.26573]
 [-196.03487]
 [-181.68422]
 [-228.99953]
 [-186.91292]
 [-218.49033]
 [-224.10962]
 [-200.53261]
 [-213.18509]
 [-211.34076]
 [-203.90186]
 [-196.25993]
 [-236.11162]]
10 Cost:  14.790597 
Prediction:
 [[149.03455]
 [182.9414 ]
 [178.00365]
 [195.01715]
 [139.93686]
 [106.23562]
 [147.27197]
 [110.68133]
 [173.70634]
 [164.36249]
 [141.84784]
 [142.26254]
 [182.54681]
 [150.45943]
 [148.71667]
 [186.49452]
 [143.07559]
 [176.21248]
 [173.26334]
 [155.0224 ]
 [172.49052]
 [172.77664]
 [163.8107 ]
 [147.22386]
 [188.23695]]
20 Cost:  8.425223 
Prediction:
 [[151.39392 ]
 [185.76611 ]
 [180.79265 ]
 [198.05373 ]
 [142.08911 ]
 [107.819786]
 [149.5468  ]
 [112.370674]
 [176.34264 ]
 [166.8015  ]
 [144.03708 ]
 [144.41599 ]
 [185.4289  ]
 [152.85571 ]
 [151.00021 ]
 [189.36115 ]

In [17]:
# Ask my score
print("Your score will be ", sess.run(hypothesis, feed_dict={X: [[100, 70, 101]]}))

Your score will be  [[190.41243]]


In [18]:
print("Other scores will be ", sess.run(hypothesis, feed_dict={X: [[60, 70, 110], [90, 100, 80]]}))

Other scores will be  [[179.61284]
 [174.21695]]


## tf.train.batch

In [32]:
filename_queue = tf.train.string_input_producer(
    ['data-01-test-score.csv'], shuffle=False, name='filename-queue') 

In [33]:
reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

In [34]:
# Default values, in case of empty columns. Also specifies the type of the decoded result
record_defaults = [[0.], [0.], [0.], [0.]]
xy = tf.decode_csv(value, record_defaults=record_defaults)

In [35]:
# collect batches of csv in
train_x_batch, train_y_batch = tf.train.batch([xy[0:-1], xy[-1:]], batch_size=10)

In [36]:
X = tf.placeholder(tf.float32, shape=[None, 3])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([3, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

In [37]:
hypothesis = tf.matmul(X, W) + b

cost = tf.reduce_mean(tf.square(hypothesis - Y))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

In [38]:
# Launch the graph in a session
sess = tf.Session()

#Initializes global variables in the graph
sess.run(tf.global_variables_initializer())

In [39]:
# start populating the filename queue
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

for step in range(2001):
    x_batch, y_batch = sess.run([train_x_batch, train_y_batch])
    cost_val, hy_val, _ = sess.run(
        [cost, hypothesis, train],
        feed_dict={X: x_batch, Y: y_batch})
    if step % 10 == 0:
        print(step, "Cost: ", cost_val,
              "\nPrediction:\n", hy_val)
        
coord.request_stop()
coord.join(threads)

0 Cost:  110227.875 
Prediction:
 [[-160.59647]
 [-205.55478]
 [-196.16599]
 [-214.26178]
 [-159.06335]
 [-119.08461]
 [-157.31792]
 [-112.11335]
 [-195.45392]
 [-183.11678]]
10 Cost:  59.87342 
Prediction:
 [[157.9099 ]
 [177.37099]
 [181.07822]
 [196.60779]
 [132.98927]
 [ 96.2807 ]
 [150.234  ]
 [115.88599]
 [162.46994]
 [148.46622]]
20 Cost:  46.76055 
Prediction:
 [[160.03348]
 [179.97244]
 [183.61543]
 [199.38037]
 [134.97775]
 [ 97.76695]
 [152.30959]
 [117.42352]
 [164.92406]
 [150.76451]]
30 Cost:  46.35796 
Prediction:
 [[160.02214]
 [180.00731]
 [183.62404]
 [199.39903]
 [135.00874]
 [ 97.80977]
 [152.32372]
 [117.43282]
 [164.97923]
 [150.84044]]
40 Cost:  46.010517 
Prediction:
 [[159.9965 ]
 [180.0248 ]
 [183.61562]
 [199.3991 ]
 [135.02646]
 [ 97.8427 ]
 [152.32393]
 [117.43178]
 [165.01804]
 [150.9011 ]]
50 Cost:  45.66636 
Prediction:
 [[159.97092]
 [180.04211]
 [183.60716]
 [199.39905]
 [135.04404]
 [ 97.87544]
 [152.32407]
 [117.43066]
 [165.0566 ]
 [150.9614 ]]
60 C